# Setup

In [ ]:
import pandapower.networks as pnet
import pandas as pd
import numpy as np
import scipy as sp
import cvxpy as cp
import seaborn as sns
import mlflow

from scipy import sparse
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(1, '..')

from src.models.matrix_operations import make_real_vector, vectorize_matrix
from src.simulation.noise import add_polar_noise_to_measurement
from src.models.regression import ComplexRegression, ComplexLasso
from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.simulation.load_profile import generate_gaussian_load
from src.simulation.network import add_load_power_control, make_y_bus
from src.simulation.simulation import run_simulation, get_current_and_voltage
from src.identification.error_metrics import error_metrics, fro_error
from src.models.noise_transformation import average_true_noise_covariance,exact_noise_covariance

# Network simulation

In [ ]:
mlflow.set_experiment('Full network with polar noise')

In [ ]:
net = pnet.create_kerber_landnetz_freileitung_2()
nodes = net.bus.shape[0]
steps = 400
load_cv = 0.2
current_magnitude_sd = 0.000001
voltage_magnitude_sd = 0.0001
phase_sd = 0.0001

In [ ]:
np.random.seed(11)
load_p, load_q = generate_gaussian_load(net.load.p_mw, net.load.q_mvar, load_cv, steps)
controlled_net = add_load_power_control(net, load_p, load_q)
sim_result = run_simulation(controlled_net, verbose=False)
y_bus = make_y_bus(controlled_net)
voltage, current = get_current_and_voltage(sim_result, y_bus)
controlled_net.bus

In [ ]:
voltage = voltage[:, 1:]
#print(y_bus)
y_v = y_bus[1,0]
y_bus = y_bus[1:, 1:]
y_bus[0,0] = y_bus[0,0] + y_v
current = np.array(voltage @ y_bus)
noisy_voltage = add_polar_noise_to_measurement(voltage, voltage_magnitude_sd, phase_sd)
noisy_current = add_polar_noise_to_measurement(current, current_magnitude_sd, phase_sd)
voltage_error, current_error = noisy_voltage - voltage, noisy_current - current

In [ ]:
np.linalg.svd(voltage, compute_uv=False)

In [ ]:
np.mean(np.abs(current), 0)

In [ ]:
mlflow_params = {
    'nodes': noisy_voltage.shape[1],
    'steps': steps, 
    'load_cv': load_cv, 
    'current_magnitude_sd': current_magnitude_sd, 
    'voltage_magnitude_sd': voltage_magnitude_sd,
    'phase_sd': phase_sd
}

# OLS Identification

In [ ]:
with mlflow.start_run(run_name='OLS'):
    ols = ComplexRegression()
    ols.fit(noisy_voltage, noisy_current)
    y_ols = ols.fitted_admittance_matrix
    ols_metrics = error_metrics(y_bus, y_ols)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(ols_metrics.__dict__)
ols_metrics

# Lasso Identification

In [ ]:
with mlflow.start_run(run_name='Lasso'):
    lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-12, -1, 40), solver=cp.GUROBI)
    lasso.fit(noisy_voltage, noisy_current)
    y_lasso = lasso.fitted_admittance_matrix
    lasso_metrics = error_metrics(y_bus, y_lasso)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(lasso_metrics.__dict__)
best_lambda = lasso.best_trial.hyperparameters['lambda']
lasso_metrics

In [ ]:
best_lambda

# TLS Identification

In [ ]:
with mlflow.start_run(run_name='TLS'):
    tls = TotalLeastSquares()
    tls.fit(noisy_voltage, noisy_current)
    y_tls = tls.fitted_admittance_matrix
    tls_metrics = error_metrics(y_bus, y_tls)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(tls_metrics.__dict__)
tls_metrics

# L1 Regularized TLS

normalizing currents

In [ ]:
with mlflow.start_run(run_name='S-TLS with covariance'):
    max_iterations = 1500
    abs_tol = 0.0000001
    rel_tol = 10e-15
    solver = cp.GUROBI
    use_cov_matrix = True
    pen_degree=1.0
    tls_weights_adaptive = np.divide(1.0, np.power(np.abs(make_real_vector(vectorize_matrix(y_tls))), 1.0))
    
    
    sigma_voltage = average_true_noise_covariance(noisy_voltage, voltage_magnitude_sd, phase_sd)
    sigma_current = average_true_noise_covariance(noisy_current, current_magnitude_sd, phase_sd)
    
    #sigma_voltage = exact_noise_covariance(voltage, voltage_magnitude_sd, phase_sd)
    #sigma_current = exact_noise_covariance(current, current_magnitude_sd, phase_sd)

    inv_sigma_current = sparse.linalg.inv(sigma_current)
    inv_sigma_voltage = sparse.linalg.inv(sigma_voltage)
    
    sparse_tls_cov = SparseTotalLeastSquare(lambda_value=0*100, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
    sparse_tls_cov.l1_multiplier_step_size = 0*0.5
    sparse_tls_cov.cons_multiplier_step_size = 0.01
    #sparse_tls_cov.set_prior(make_real_vector(vectorize_matrix(np.zeros(y_tls.shape))), SparseTotalLeastSquare.LAPLACE,
    #                         np.diag(tls_weights_adaptive))

    sparse_tls_cov.l1_target = 2*np.count_nonzero(y_bus)
    sparse_tls_cov.l1_target = 1.0*(np.sum(np.power(np.abs(np.real(y_bus)),pen_degree))
                                    + np.sum(np.power(np.abs(np.imag(y_bus)),pen_degree)))
    sparse_tls_cov.l1_target = 1.0*2*np.sum(np.abs(make_real_vector(np.diag(y_tls))))

    sparse_tls_cov.fit(noisy_voltage, noisy_current, inv_sigma_voltage, inv_sigma_current, y_init=y_tls)
    
    y_sparse_tls_cov = sparse_tls_cov.fitted_admittance_matrix
    sparse_tls_cov_metrics = error_metrics(y_bus, y_sparse_tls_cov)
    
    sparse_tls_cov_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_tls_cov.iterations])
    sparse_tls_cov_targets = pd.Series([i.target_function for i in sparse_tls_cov.iterations])
    
    mlflow.log_param('lambda', best_lambda)
    mlflow.log_param('max_iterations', max_iterations)
    mlflow.log_param('abs_tol', abs_tol)
    mlflow.log_param('rel_tol', rel_tol)
    mlflow.log_param('solver', solver)
    mlflow.log_param('use_cov_matrix', use_cov_matrix)
    mlflow.log_params(mlflow_params)
    mlflow.log_metrics(sparse_tls_cov_metrics.__dict__)

    for i in range(len(sparse_tls_cov_errors)):
        mlflow.log_metric('fro_error_evo', value=sparse_tls_cov_errors[i], step=i)
        mlflow.log_metric('opt_cost_evo', value=sparse_tls_cov_targets[i], step=i)

In [ ]:
sparse_tls_cov_errors.plot()
#sparse_tls_cov_targets.copy().multiply(0.00004).plot()


In [ ]:
sparse_tls_cov_targets.plot()

In [ ]:
plt.plot(sparse_tls_cov.tmp)

# Result analysis

In [ ]:
sns.heatmap(np.abs(y_bus));

In [ ]:
sns.heatmap(np.abs(y_bus - y_ols));

In [ ]:
sns.heatmap(np.abs(y_bus - y_lasso));

In [ ]:
sns.heatmap(np.abs(y_bus - y_tls),vmin=0,vmax=6);

In [ ]:
sns.heatmap(np.abs(y_bus - y_sparse_tls_cov),vmin=0,vmax=6);

In [ ]:
sns.heatmap(np.abs(y_sparse_tls_cov));